In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import time
import os

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# base_url = 'https://en.zalando.de'
def get_url(url=''):
    if base_url not in url:
        url = f"{base_url}{url}"
    return BeautifulSoup(requests.get(
        url,
        verify=False,
        timeout=5,
        headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'en-US,en;q=0.9',
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        }
    ).content, 'html.parser')

# Zalando

In [4]:
# home = get_url('/women-home/')

In [5]:
# soup = BeautifulSoup(home.content, 'html.parser')

In [6]:
# main_links = {}

# for li in soup.find_all('nav')[2].find_all('li'):
#     main_links[li.get('aria-label')] = li.a.get('href')

In [7]:
# main_links

In [8]:
# cats = ['Clothing', 'Shoes', 'Sports', 'Accessories']

In [9]:
# c = cats[0]

# HnM

In [42]:
base_url = 'https://www2.hm.com'
filename_format = '../data/HnM_{}.parquet'
img_filename = '../images/HnM_{}{}'

### Getting categories for Women

In [11]:
top_cat = 'Women'

In [12]:
main = get_url('/de_de/damen.html')

In [13]:
cur_idx = 0

In [14]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

for li in main.find_all('nav')[2].find_all('ul')[0].find_all('ul')[3].find_all('li'):
    li_name = li.a.get_text()[4:].strip()
    if 'Alles zeigen' in li_name:
        continue
    names.append(li_name)
    urls.append(li.a.get('href'))
    parents.append(None)
    main_cats.append(cur_idx)
    cur_idx += 1

Getting sub-categories

In [15]:
# lists to store all filter names and values
# filters are extracted at parent category level
filter_names = []
filter_options = []
filter_parents = []

get_filter_options = lambda x: (x.find('span', attrs={'class': 'text'}).text, f"{x.find('input').get('data-name')}={x.find('input').get('value')}")

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    for li in cat_html.find_all('nav')[2].find_all('ul')[4].find_all('li'):
        if li.a.get('href') == urls[i]:
            for sub_li in li.ul.find_all('li'):
                li_name = sub_li.a.get_text()[4:].strip()
                if 'Alles zeigen' in li_name:
                    continue
                names.append(li_name)
                urls.append(sub_li.a.get('href'))
                parents.append(i)
                
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class':'filter-levels'})[0].ul.contents

    for f in filters_:
        if f != '\n':
            options = list(map(get_filter_options, f.ul.find_all('li')))
            filter_options.extend(options)
            filter_names.extend([f.find(attrs={'class':'filter-name'}).text.strip()]*len(options))
            filter_parents.extend([c]*len(options))


Creating dataframe of categories and sub-categories

In [16]:
if os.path.exists(filename_format.format('Categories')):
    cats = pd.read_parquet(filename_format.format('Categories'))
else:
    cats = pd.DataFrame(columns=['Category', 'URL', 'Parent_Category', 'Top_Category', 'Skip'])
    
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})])

In [17]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False


In [18]:
cats.shape

(225, 5)

### Getting categories for Men

In [19]:
top_cat = 'Men'

In [20]:
main = get_url('/de_de/herren.html')

In [21]:
cur_idx = cats.index[-1]+1

In [22]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

for li in main.find_all('nav')[2].find_all('ul')[0].find_all('ul')[3].find_all('li'):
    li_name = li.a.get_text()[4:].strip()
    if 'Alles zeigen' in li_name:
        continue
    names.append(li_name)
    urls.append(li.a.get('href'))
    parents.append(None)
    main_cats.append(cur_idx)
    cur_idx += 1

Getting sub-categories

In [23]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: (x.find('span', attrs={'class': 'text'}).text, f"{x.find('input').get('data-name')}={x.find('input').get('value')}")

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    for li in cat_html.find_all('nav')[2].find_all('ul')[4].find_all('li'):
        if li.a.get('href') == urls[i]:
            for sub_li in li.ul.find_all('li'):
                li_name = sub_li.a.get_text()[4:].strip()
                if 'Alles zeigen' in li_name:
                    continue
                names.append(li_name)
                urls.append(sub_li.a.get('href'))
                parents.append(c)
                
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class':'filter-levels'})[0].ul.contents

    for f in filters_:
        if f != '\n':
            options = list(map(get_filter_options, f.ul.find_all('li')))
            filter_options.extend(options)
            filter_names.extend([f.find(attrs={'class':'filter-name'}).text.strip()]*len(options))
            filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [24]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [25]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False


In [26]:
cats.shape

(337, 5)

### Getting categories for Baby

In [27]:
top_cat = 'Baby'

In [28]:
main = get_url('/de_de/baby.html')

In [29]:
cur_idx = cats.index[-1]+1

In [30]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

for li in main.find_all('nav')[2].find_all('ul')[0].find_all('ul')[3].find_all('li'):
    li_name = li.a.get_text()[4:].strip()
    if 'Alles zeigen' in li_name:
        continue
    names.append(li_name)
    urls.append(li.a.get('href'))
    parents.append(None)
    main_cats.append(cur_idx)
    cur_idx += 1

Getting sub-categories

In [31]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: (x.find('span', attrs={'class': 'text'}).text, f"{x.find('input').get('data-name')}={x.find('input').get('value')}")

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    for li in cat_html.find_all('nav')[2].find_all('ul')[4].find_all('li'):
        if li.a.get('href') == urls[i]:
            for sub_li in li.ul.find_all('li'):
                li_name = sub_li.a.get_text()[4:].strip()
                if 'Alles zeigen' in li_name:
                    continue
                names.append(li_name)
                urls.append(sub_li.a.get('href'))
                parents.append(c)
                
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class':'filter-levels'})[0].ul.contents

    for f in filters_:
        if f != '\n':
            options = list(map(get_filter_options, f.ul.find_all('li')))
            filter_options.extend(options)
            filter_names.extend([f.find(attrs={'class':'filter-name'}).text.strip()]*len(options))
            filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [32]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [33]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False


In [34]:
cats.shape

(365, 5)

### Getting categories for Kids

In [35]:
top_cat = 'Kids'

In [36]:
main = get_url('/de_de/kinder.html')

In [37]:
cur_idx = cats.index[-1]+1

In [38]:
# lists to store all categories and sub-categories
names = []
urls = []
parents = []
main_cats = []

for li in main.find_all('nav')[2].find_all('ul')[0].find_all('ul')[3].find_all('li'):
    li_name = li.a.get_text()[4:].strip()
    if 'Alles zeigen' in li_name:
        continue
    names.append(li_name)
    urls.append(li.a.get('href'))
    parents.append(None)
    main_cats.append(cur_idx)
    cur_idx += 1

Getting sub-categories

In [39]:
# lists to store all filter names and values
# filters are extracted at parent category level
get_filter_options = lambda x: (x.find('span', attrs={'class': 'text'}).text, f"{x.find('input').get('data-name')}={x.find('input').get('value')}")

for i, c in enumerate(main_cats):
    cat_html = get_url(urls[i])
    
    for li in cat_html.find_all('nav')[2].find_all('ul')[4].find_all('li'):
        if li.a.get('href') == urls[i]:
            for sub_li in li.ul.find_all('li'):
                li_name = sub_li.a.get_text()[4:].strip()
                if 'Alles zeigen' in li_name:
                    continue
                names.append(li_name)
                urls.append(sub_li.a.get('href'))
                parents.append(c)
                
    # getting all filters
    filters_ = cat_html.find_all('div', attrs={'class':'filter-levels'})[0].ul.contents

    for f in filters_:
        if f != '\n':
            options = list(map(get_filter_options, f.ul.find_all('li')))
            filter_options.extend(options)
            filter_names.extend([f.find(attrs={'class':'filter-name'}).text.strip()]*len(options))
            filter_parents.extend([c]*len(options))


Updating dataframe of categories and sub-categories

In [40]:
cats = pd.concat([cats, pd.DataFrame({
    'Category': names,
    'URL': urls,
    'Parent_Category': parents,
    'Top_Category': [top_cat]*len(names),
    'Skip': [False]*len(names), # to allow skipping later
})]).reset_index().drop(columns='index')

In [41]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False


In [42]:
cats.shape

(407, 5)

### Removing categories which we dont want

In [43]:
cats.groupby('Top_Category').count()

,Category,URL,Parent_Category,Skip
Top_Category,,,,
Baby,28,28,21,28
Kids,42,42,34,42
Men,112,112,91,112
Women,225,225,196,225


In [44]:
cats[cats['Parent_Category'].isna()]['Category'].unique()

array(['Shirts & Tops', 'Basics', 'Hemden & Blusen',
       'Cardigans & Pullover', 'Strickwaren', 'Blazer', 'Jacken & Mäntel',
       'Hosen', 'Jeans', 'Shorts', 'Röcke', 'Kleider',
       'Jumpsuits & Playsuits', 'Schuhe', 'Accessoires', 'Bademode',
       'Unterwäsche', 'Loungewear', 'Nachtwäsche',
       'Socken & Strumpfhosen', 'Sportbekleidung', 'Umstandsmode',
       'Große Größen', 'Beauty', 'Hundebekleidung und -Accessoires',
       'Hoodies & Sweatshirts', 'Pflegeprodukte', 'Petite-Kleidung',
       'Premium Selection', 'Strickkleidung', 'Hemden', 'T-Shirts',
       'Socken', 'Blazer & Anzüge', 'Sportmode', 'Alle anzeigen',
       'Neuheiten', 'Kleidung', 'Outdoor', 'Kostüme'], dtype=object)

In [45]:
cats_to_delete = [
    'Umstandsmode', 'Große Größen', 'Hundebekleidung und -Accessoires', 'Pflegeprodukte', 
    'Petite-Kleidung', 'Premium Selection', 'Alle anzeigen', 'Neuheiten'
]

In [46]:
idx_delete = cats[(cats['Category'].isin(cats_to_delete)) & (cats['Parent_Category'].isna())].index
idx_delete

Int64Index([21, 22, 24, 26, 27, 28, 236, 244, 245, 337, 338, 365, 366], dtype='int64')

In [47]:
# delete parent categories
cats.drop(labels=idx_delete, axis=0, inplace=True)

#delete the children
cats.drop(labels = cats[cats['Parent_Category'].isin(idx_delete)].index, axis=0, inplace=True)

In [48]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False


In [49]:
cats.shape

(348, 5)

In [50]:
# Check count of parent categories and child categories per top category\
cats.groupby('Top_Category').count()

,Category,URL,Parent_Category,Skip
Top_Category,,,,
Baby,26,26,21,26
Kids,40,40,34,40
Men,102,102,84,102
Women,180,180,157,180


In [53]:
cats[(cats['Top_Category'] == 'Kids') & (~cats['Parent_Category'].isna())].head()

,Category,URL,Parent_Category,Top_Category,Skip
373,Kleider,/de_de/kinder/produkte/kleidung/kleider.html,367.0,Kids,False
374,Oberteile & T-Shirts,/de_de/kinder/produkte/kleidung/oberteile-t-sh...,367.0,Kids,False
375,Pullover & Sweatshirts,/de_de/kinder/produkte/kleidung/pullover-sweat...,367.0,Kids,False
376,Hosen & Jeans,/de_de/kinder/produkte/kleidung/hosen-jeans.html,367.0,Kids,False
377,Röcke,/de_de/kinder/produkte/kleidung/roecke.html,367.0,Kids,False


In [54]:
# saving categories
cats.to_parquet(filename_format.format('Categories'))

Creating dataframe of fitler options per category

In [55]:
filters = pd.DataFrame({
    'Parent_Category': filter_parents,
    'Filter_Name': filter_names,
    'Filter_Option': filter_options,
})

In [56]:
filters.shape

(6733, 3)

In [57]:
filters_t = filters[filters['Parent_Category'].isin(cats[cats['Parent_Category'].isna()].index)]

In [58]:
filters_t.head(10)

,Parent_Category,Filter_Name,Filter_Option
0,0,Sortieren nach,"(Empfohlen, sort=stock)"
1,0,Sortieren nach,"(Neuheiten, sort=newProduct)"
2,0,Sortieren nach,"(Niedrigster Preis, sort=ascPrice)"
3,0,Sortieren nach,"(Höchster Preis, sort=descPrice)"
4,0,Conscious choice,"(Conscious choice, sustainabilities=Conscious+..."
5,0,Marke,"(Fila, brandNames=Fila)"
6,0,Marke,"(H&M, brandNames=H%26M)"
7,0,Marke,"(Minga London, brandNames=Minga+London)"
8,0,Marke,"(Superdry, brandNames=Superdry)"
9,0,Marke,"(aim'n, brandNames=aim%27n)"


In [59]:
filters_t.shape

(5029, 3)

In [60]:
filters_t['Filter_Name'].unique()

array(['Sortieren nach', 'Conscious choice', 'Marke', 'Größe', 'Farbe',
       'MUSTER', 'Produkttyp', 'Style', 'Funktion', 'Ausschnitt',
       'Ärmellänge', 'Länge', 'Ärmelstil', 'Kragen', 'Passform', 'Anlass',
       'Konzept', 'Qualität', 'Material', 'Bundhöhe', 'Schuhstil',
       'Absatzhöhe', 'Accessoirestil', 'Deckkraft', 'Finish', 'Hautton',
       'Hauttyp', 'Duftfamilie', 'Nutzen', 'Problemstelle', 'Form',
       'Hauptinhaltsstoff', 'LSF', 'Kundengruppe'], dtype=object)

In [61]:
filters_to_keep = [
    'Farbe', 'MUSTER', 'Produkttyp', 'Style', 'Funktion', 'Ausschnitt', 'Ärmellänge', 
    'Länge', 'Ärmelstil', 'Kragen', 'Passform', 'Anlass', 'Konzept', 'Bundhöhe', 'Schuhstil', 
    'Absatzhöhe', 'Accessoirestil', 'Deckkraft', 'Finish', 'Hautton'
]

In [62]:
filters_t = filters_t[filters_t['Filter_Name'].isin(filters_to_keep)]

In [63]:
filters_t.shape

(3261, 3)

In [64]:
filters_t.head()

,Parent_Category,Filter_Name,Filter_Option
31,0,Farbe,"(Beige, colorWithNames=beige_f5f5dc)"
32,0,Farbe,"(Blau, colorWithNames=blau_0000ff)"
33,0,Farbe,"(Braun, colorWithNames=braun_a52a2a)"
34,0,Farbe,"(Gelb, colorWithNames=gelb_ffff00)"
35,0,Farbe,"(Grau, colorWithNames=grau_808080)"


In [65]:
filters_t.groupby('Parent_Category').count()

,Filter_Name,Filter_Option
Parent_Category,,
0,103,103
1,115,115
2,91,91
3,63,63
4,62,62
5,43,43
6,71,71
7,85,85
8,51,51


In [66]:
# saving filters
filters_t.to_parquet(filename_format.format('Filters'))

Iterate through all sub-categories - get all products (images & ids)

In [31]:
# lamda functions for getting various params
img_url = lambda x: x.find_all('img')[0].get('src') if x.find_all('img')[0].get('src') else x.find_all('img')[0].get('data-src')
img_ext = lambda x: re.search('set=source\[[a-z0-9/]*(\.[a-z]*)]', img_url(x))[1]
item_name = lambda x: x.find_all('a')[0].get('title')
item_url = lambda x: x.find_all('a')[0].get('href')
item_code = lambda x: x.get('data-articlecode')

def save_image(code, img, ext, verbose=False):
    target = img_filename.format(code, ext)
#     target = f"../images/{code}{ext}"
    if os.path.exists(target):
#         print("Already there")
        return 0
    if verbose:
        print(f"Img not found - {code}.", end=' ')
    img_data = requests.get(img if img[:6] == 'https:' else f"https:{img}")
    if img_data.status_code != 200:
        return 0
    with open(target, 'wb') as f:
        f.write(img_data.content)
        if verbose:
            print(f"Downloaded.")
        return 1

In [76]:
skip = [79, 279]
cats.loc[skip, 'Skip'] = True

In [77]:
size = 1000

if os.path.exists(filename_format.format('Items')):
    df_items = pd.read_parquet(filename_format.format('Items'))
else:
    df_items = pd.DataFrame(columns=['Code', 'Name', 'URL', 'Img_URL', 'Img_Ext', 'Category'])

i = 0
total = cats[~cats['Parent_Category'].isna()].shape[0]

print(f'Total Categories = {total}')
               
for idx, row in cats[(~cats['Parent_Category'].isna()) & (cats['Skip'] == False)].iterrows():
    if df_items[df_items['Category'] == idx].shape[0] > 0:
        # already completed,
        i += 1
        print(f"Skipping {idx} - {row['Top_Category']} - {row['Category']}")
        continue
    
    subcat = get_url(f"{row['URL']}?image=stillLife&offset=0&page-size={size}")
    all_items = subcat.find_all('article')
#     print(f"Found {len(all_items)} items.")
    
    codes = list(map(item_code, all_items))
    img_urls = list(map(img_url, all_items))
    img_exts = list(map(img_ext, all_items))
    
    img_saved = sum(list(map(save_image, codes, img_urls, img_exts)))
    
    df_items = pd.concat([df_items, pd.DataFrame({
        'Code': codes,
        'Name': list(map(item_name, all_items)),
        'URL': list(map(item_url, all_items)),
        'Img_URL': img_urls,
        'Img_Ext': img_exts,
        'Category': [idx]*len(all_items)
    })], ignore_index=True)
    
    i += 1
    print(f"Completed {idx} - {row['Category']} - Saved {len(all_items)} items. Images saved = {img_saved}.")
    if i%10 == 0:
        df_items.to_parquet(filename_format.format('Items'))
        time.sleep(1)

df_items.to_parquet(filename_format.format('Items'))
print("All Done!")

Total Categories = 296
Skipping 29 - Women - Cut-Out-Tops
Completed 30 - Bralette-Tops - Saved 0 items. Images saved = 0.
Skipping 31 - Women - Bandeau-Tops
Skipping 32 - Women - Oberteile mit Puffärmeln
Skipping 33 - Women - Trägertops & Tanktops
Skipping 34 - Women - Kurzarm-Shirts
Skipping 35 - Women - Langarm-Shirts
Skipping 36 - Women - Crop-Tops
Skipping 37 - Women - Bodys
Skipping 38 - Women - Off-Shoulder Tops
Skipping 39 - Women - T-Shirts
Skipping 40 - Women - T-Shirts bedruckt
Skipping 41 - Women - Oberteile
Skipping 42 - Women - Cardigans & Pullover
Skipping 43 - Women - Kleider & Röcke
Skipping 44 - Women - Hosen & Leggings
Skipping 45 - Women - Leinenshirts
Skipping 46 - Women - Hemden
Skipping 47 - Women - Blusen
Skipping 48 - Women - Tuniken
Skipping 49 - Women - Off-Shoulder-Shirts
Skipping 50 - Women - Jeanshemden
Skipping 51 - Women - Pullunder
Skipping 52 - Women - Cardigans
Skipping 53 - Women - Pullover
Completed 54 - Ponchos - Saved 0 items. Images saved = 0.
Ski

Completed 305 - Sandalen & Espadrilles - Saved 24 items. Images saved = 0.
Completed 306 - Boots - Saved 12 items. Images saved = 0.
Completed 307 - Elegante Schuhe - Saved 10 items. Images saved = 0.
Completed 308 - Loafer - Saved 14 items. Images saved = 0.
Completed 309 - Alltagsmasken - Saved 0 items. Images saved = 0.
Completed 310 - Schals - Saved 17 items. Images saved = 0.
Completed 311 - Krawatten, Fliegen & Einstecktücher - Saved 12 items. Images saved = 0.
Completed 312 - Gürtel & Hosenträger - Saved 16 items. Images saved = 0.
Completed 313 - Taschen - Saved 42 items. Images saved = 0.
Completed 314 - Sonnenbrillen - Saved 60 items. Images saved = 0.
Completed 315 - Schmuck - Saved 62 items. Images saved = 0.
Completed 316 - Sportzubehör - Saved 21 items. Images saved = 0.
Completed 317 - Sonstiges - Saved 2 items. Images saved = 0.
Completed 318 - Hüte & Caps - Saved 93 items. Images saved = 0.
Completed 319 - Handschuhe - Saved 12 items. Images saved = 0.
Completed 320 - 

In [78]:
df_items

,Code,Name,URL,Img_URL,Img_Ext,Category
0,1071498001,Cut-out-Shirt,/de_de/productpage.1071498001.html,//lp2.hm.com/hmgoepprod?set=source[/a2/90/a290...,.jpg,29.0
1,1069183001,Kurzes Shirt mit Cut-out,/de_de/productpage.1069183001.html,//lp2.hm.com/hmgoepprod?set=source[/12/09/1209...,.jpg,29.0
2,0985757001,Kurzshirt mit Cut-out,/de_de/productpage.0985757001.html,//lp2.hm.com/hmgoepprod?set=source[/68/a3/68a3...,.jpg,29.0
3,0985757002,Kurzshirt mit Cut-out,/de_de/productpage.0985757002.html,//lp2.hm.com/hmgoepprod?set=source[/10/22/1022...,.jpg,29.0
4,1069183002,Kurzes Shirt mit Cut-out,/de_de/productpage.1069183002.html,//lp2.hm.com/hmgoepprod?set=source[/d1/3a/d13a...,.jpg,29.0
...,...,...,...,...,...,...
22758,0897909012,Weiche Hausschuhe,/de_de/productpage.0897909012.html,//lp2.hm.com/hmgoepprod?set=source[/3d/04/3d04...,.jpg,406.0
22759,1033466002,Weiche Hausschuhe mit Applikationen,/de_de/productpage.1033466002.html,//lp2.hm.com/hmgoepprod?set=source[/23/2e/232e...,.jpg,406.0
22760,0897909013,Weiche Hausschuhe,/de_de/productpage.0897909013.html,//lp2.hm.com/hmgoepprod?set=source[/cd/b5/cdb5...,.jpg,406.0
22761,0897909011,Weiche Hausschuhe,/de_de/productpage.0897909011.html,//lp2.hm.com/hmgoepprod?set=source[/bc/b0/bcb0...,.jpg,406.0


In [79]:
pd.merge(
    df_items, 
    cats[['Category', 'Parent_Category']], 
    left_on='Category', 
    right_index=True, 
    how='left').groupby('Parent_Category').agg({
        'Code':['count', 'nunique']
    })

Code        
                count nunique
Parent_Category              
0.0              1414     867
1.0               671     671
2.0               418     379
3.0               351     304
6.0               427     305
7.0               914     668
8.0               681     467
9.0               318     240
10.0              330     155
11.0             1442     740
12.0               44      44
13.0              351     267
14.0              753     735
15.0              235     235
16.0             1432     906
18.0              260     260
19.0              173     160
20.0              591     425
23.0              725     669
25.0              340     340
225.0             245     137
226.0             220     220
227.0             146     130
229.0             199     160
230.0             593     449
231.0             278     262
232.0             120     108
233.0              90      90
234.0              20      20
235.0              16      16
237.0             137      63
238.0             131     125
239.0             337     320
240.0             227     227
241.0             315     240
243.0              98      76
339.0            1568    1075
340.0              71      63
341.0             136     125
342.0              68      68
367.0            5031    4249
368.0             323     295
369.0             309     296
370.0             215     215

In [80]:
df_items.to_parquet(filename_format.format('Items'))

In [82]:
# Get any failed images
i = 0
for idx, row in df_items.iterrows():
    res = save_image(row['Code'], row['Img_URL'], row['Img_Ext'], verbose=True)
    if res > 0:
        i += 1
        if i%10 == 0:
            time.sleep(1)

In [83]:
# Unique item count
df_items['Code'].nunique()

16473

In [84]:
df_items['Category'].unique()

array([ 29.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
        41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,
        52.,  53.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,
        64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,
        75.,  76.,  77.,  78.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,
        87.,  88.,  89.,  90.,  91.,  92.,  93.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       133., 135., 136., 137., 138., 139., 140., 141., 142., 143., 144.,
       145., 146., 147., 148., 149., 150., 151., 152., 153., 154., 155.,
       156., 157., 158., 159., 160., 161., 162., 163., 164., 165., 166.,
       167., 168., 169., 170., 203., 204., 205., 206., 207., 208., 209.,
       210., 211., 212., 213., 215., 216., 217., 24

Iterate through each filter for each sub-category - get all product ids and tag them

In [89]:
size = 10000

if os.path.exists(filename_format.format('FilterAttributes')):
    df_attributes = pd.read_parquet(filename_format.format('FilterAttributes'))
else:
    df_attributes = pd.DataFrame(columns=['Code', 'Filter_ID'])

i = 0
total = filters_t.shape[0]

print(f'Total Filters = {total}')

for idx, row in filters_t.iterrows():
    if df_attributes[df_attributes['Filter_ID'] == idx].shape[0] > 0:
        # already completed,
        i += 1
        if i%10 == 0:        
            print(f"Skipped {i}")
        continue
    
    cat_filter = get_url(f"{cats.loc[int(row['Parent_Category'])]['URL']}?image=stillLife&offset=0&page-size={size}&{row['Filter_Option'][1]}")
    codes = list(map(item_code, cat_filter.find_all('article')))
    df_attributes = pd.concat([
        df_attributes,
        pd.DataFrame({
            'Code': codes,
            'Filter_ID': [idx]*len(codes)
        })
    ])
    
    i += 1
    if i%10 == 0:
        df_attributes.to_parquet(filename_format.format('FilterAttributes'))
        print(f"Completed {i}")
        time.sleep(1)

df_attributes.to_parquet(filename_format.format('FilterAttributes'))
print("All Done!")

Total Filters = 3261
Skipped 10
Skipped 20
Skipped 30
Skipped 40
Skipped 50
Skipped 60
Skipped 70
Skipped 80
Skipped 90
Skipped 100
Skipped 110
Skipped 120
Skipped 130
Skipped 140
Skipped 150
Skipped 160
Skipped 170
Skipped 180
Skipped 190
Skipped 200
Skipped 210
Skipped 220
Skipped 230
Skipped 240
Skipped 250
Skipped 260
Skipped 270
Skipped 280
Skipped 290
Skipped 300
Skipped 310
Skipped 320
Skipped 330
Skipped 340
Skipped 350
Skipped 360
Skipped 370
Skipped 380
Skipped 390
Skipped 400
Skipped 410
Skipped 420
Skipped 430
Skipped 440
Skipped 450
Skipped 460
Skipped 470
Skipped 480
Skipped 490
Skipped 500
Skipped 510
Skipped 520
Skipped 530
Skipped 540
Skipped 550
Skipped 560
Skipped 570
Skipped 580
Skipped 590
Skipped 600
Skipped 610
Skipped 620
Skipped 630
Skipped 640
Skipped 650
Skipped 660
Skipped 670
Skipped 680
Skipped 690
Skipped 700
Skipped 710
Skipped 720
Skipped 730
Skipped 740
Skipped 750
Skipped 760
Skipped 770
Skipped 780
Skipped 790
Skipped 800
Skipped 810
Skipped 820
Skip

In [90]:
df_attributes

,Code,Filter_ID
0,0979236026,31.0
1,1080623001,31.0
2,1000128007,31.0
3,1000128008,31.0
4,0903309006,31.0
...,...,...
166,0867741006,6732.0
167,0706020018,6732.0
168,1021159001,6732.0
169,0929864004,6732.0


In [91]:
df_attributes['Code'].nunique()

17394

Removing some images which would probably not be good for training

In [ ]:
# bad_images = [
#     '0111565001', '0111593001', '0156227001', '0156231001', '0158340001', '0228257001', '0228257002',
#     '0228257003', '0228257004', '0228257017', '0228257018', '0542413008', '0542413012', '0561274001',
#     '0987624001', '0987624004', '0987624005', '0987628001', '0987628002', '0987628003', '0987628004', 
#     '0987628006'
# ]